In [1]:
import pandas as pd
import joblib

# Load the trained model
model_path = "knn_learning_style_model.pkl"
knn_model = joblib.load(model_path)

# Load reference dataset to get feature structure
reference_data_path = "Processed_Grad_responses.xlsx"  # Change this if needed
reference_df = pd.read_excel(reference_data_path)

# Drop learning style labels to get correct feature structure
learning_style_columns = ["Visual_Learner", "Auditory_Learner", "Kinesthetic_Learner"]
reference_df = reference_df.drop(columns=learning_style_columns, errors="ignore")

# Get the expected feature columns from the trained model
expected_features = knn_model.feature_names_in_

# Function to preprocess new user input
def preprocess_input(user_data):
    """
    Prepares a new user response for prediction by ensuring feature consistency.
    
    :param user_data: Dictionary containing user responses.
    :return: Processed DataFrame ready for model prediction.
    """
    # Convert user input to a DataFrame
    user_df = pd.DataFrame([user_data])

    # Combine with reference data for one-hot encoding consistency
    combined_df = pd.concat([reference_df, user_df], ignore_index=True)
    combined_df = pd.get_dummies(combined_df)  # Apply one-hot encoding

    # Ensure user data has only the expected model features
    for col in expected_features:
        if col not in combined_df.columns:
            combined_df[col] = 0  # Add missing columns with default value 0

    # Drop any extra columns not expected by the model
    processed_input = combined_df[expected_features]

    # Return only the last row (user input)
    return processed_input.iloc[-1:].reset_index(drop=True)

# Function to predict learning style
def predict_learning_style(user_data):
    """
    Predicts the learning style based on user responses.
    
    :param user_data: Dictionary containing user responses.
    :return: Dictionary with learning style predictions.
    """
    # Preprocess the input
    processed_input = preprocess_input(user_data)

    # Make a prediction
    prediction = knn_model.predict(processed_input)

    # Convert prediction to readable format
    learning_styles = ["Visual Learner", "Auditory Learner", "Kinesthetic Learner"]
    predicted_styles = {learning_styles[i]: int(prediction[0][i]) for i in range(len(learning_styles))}

    return predicted_styles


In [ ]:
new_user = {
    "How often do you benefit from visual aids (e.g., diagrams, animations)?": 1,  # Rarely
    "Do you trust AI-based systems to provide personalized learning recommendations?": 0,  # No
    "Do you prefer learning at your own pace or following a structured schedule?": "Following a structured schedule",
    "How important is feedback to you while learning new material?": 2,  # Moderately important
    "What motivates you to stay engaged with learning?": "Hands-on projects and real-world applications",
}


predicted_learning_style_2 = predict_learning_style(new_user)
print("Predicted Learning Style:", predicted_learning_style_2)


Predicted Learning Style: {'Visual Learner': 0, 'Auditory Learner': 0, 'Kinesthetic Learner': 1}
